Задание:

В файле milk_promo_sales.csv приведены промо продажи по неделям одного SKU в молочной категории. 
Описание полей:
store_id	- идентификатор магазина
period_id	 - период (неделя) наблюдения
sales_volume - объем продаж

Промопериодом мы считаем непрерывный отрезок времени, когда были продажи в рамках одного магазина. Пример разбивки по промопериодам и нахождение длительности промопериода находятся в файле example.xls

Используя данные фактических продаж необходимо найти:
1. Общее количество промопериодов (во всех магазинах)
2. Медиану продолжительности промопериода (количество недель)
3. Объем  продаж по каждому промопериоду
4. Медиану количества промопериодов на один магазин

In [1]:
import pandas as pd

Сначала я сортирую значения датасета, чтобы потом ввести столбец continuous_id. Для этого необходимо, чтобы period_id и store_id были упорядочены.

In [2]:
data = pd.read_csv(r'D:\Users\Ivan\Desktop\milk_promo_sales.csv')
sorted_data = data.sort_values(by=['store_id', 'period_id'])

 Добавляю в sorted_data столбец continuous_id , показывающий непрерывные промежутки времени в нашем отсортированном списке. Для этого создаю структуру Series, в которой строка будет False, если period_id этой строки ровно на 1 больше period_id предыдущей, и True в ином случае.

In [3]:
sorted_data['continuous_id'] = (sorted_data.period_id.diff().fillna(0) != 1).cumsum()

Ниже показана логика, по которой создаётся continuous_id:

```
diff  boolean  continuous  period_id
                                            
0.0     True           1        191
1.0    False           1        192
1.0    False           1        193
1.0    False           1        194
1.0    False           1        195
1.0    False           1        196
1.0    False           1        197
1.0    False           1        198
1.0    False           1        199
1.0    False           1        200
1.0    False           1        201
1.0    False           1        202
1.0    False           1        203
1.0    False           1        204
1.0    False           1        205
1.0    False           1        206
2.0     True           2        208
1.0    False           2        209
2.0     True           3        211
1.0    False           3        212
1.0    False           3        213
1.0    False           3        214
2.0     True           4        216
1.0    False           4        217
1.0    False           4        218
1.0    False           4        219
1.0    False           4        220
2.0     True           5        222
1.0    False           5        223
1.0    False           5        224
```

Так как промопериод, не только непрерывный промежуток времени, но ещё и в рамках одного магазина (а continuous_id не изменится, если period_id следующего магазина на 1 больше period_id текущего магазина) - группирую сортированный список по id непрерывных промежутков времени и по id магазинов. 

In [4]:
promos = sorted_data.groupby(['store_id', 'continuous_id'])

Для получения медианы количества промопериодов на один магазин, я группирую sorted_data по id магазина и считаю сколько уникальных непрерывных периодов времени в рамках каждого магазина.

In [8]:
print('Общее количество промопериодов:', len(promos))
print('Медиана продолжительности промопериода:', promos.size().median())
print('Медиана количества промопериодов на один магазин',
      sorted_data.groupby('store_id').continuous_id.nunique().median())
print('Объём продаж по каждому промопериоду:\n\n', promos.sales_volume.sum().head(10))

Общее количество промопериодов: 140992
Медиана продолжительности промопериода: 3.0
Медиана количества промопериодов на один магазин 6.0
Объём продаж по каждому промопериоду:

 store_id  continuous_id
4168621   1                  86.35
          2                   9.90
          3                  15.85
          4                  18.20
          5                  48.30
          6                  52.95
4168624   7                1489.30
4168636   8                   8.20
          9                  30.65
          10                 10.25
Name: sales_volume, dtype: float64
